In [1]:
!echo hello

hello


In [4]:
!pip install snowflake --user

  Created wheel for snowflake: filename=snowflake-0.0.3-py3-none-any.whl size=2967 sha256=796e855ae5cb83ea19d2cce1cd3c8ea169c57adadac66c12047f3f8a619cb222
  Stored in directory: /home/ec2-user/.cache/pip/wheels/06/07/1f/67c1209a5c6b1adf7bfac378abd403086de02d50b0a17d5fbd
Successfully built snowflake
You should consider upgrading via the '/home/ec2-user/anaconda3/envs/python3/bin/python -m pip install --upgrade pip' command.


In [1]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import math
import pandas as pd
import re
from datetime import date, datetime
## Limit Size of Returned Records
MAX_QUERY_RETURN_SIZE = 1000000

## Credentials
SF_CREDS = 'datascience-max-dev-sagemaker-notebooks'

In [2]:
import json
import snowflake.connector
from abc import ABCMeta, abstractmethod
import boto3

class Credentials(metaclass=ABCMeta):
    pass
    
    
class SSMPSCredentials(Credentials):
    def __init__(self, secretid: str):
        self._secretid = secretid
        self._secrets = {}
        
    def get_keys(self):
        """
        credential fetching 
        """
        _aws_sm_args = {'service_name': 'secretsmanager', 'region_name': 'us-east-1'}
        secrets_client = boto3.client(**_aws_sm_args)
        get_secret_value_response = secrets_client.get_secret_value(SecretId=self._secretid)
        return get_secret_value_response
    
    
class BaseConnector(metaclass=ABCMeta):
    @abstractmethod
    def connect(self):
        raise NotImplementedError
        

class SnowflakeConnector(BaseConnector):
    def __init__(self, credentials: Credentials):
        keys = credentials.get_keys()
        self._secrets = json.loads(keys.get('SecretString', "{}"))

    def connect(self, dbname: str, schema: str = 'DEFAULT'):
        ctx = snowflake.connector.connect(
            user=self._secrets['login_name'],
            password=self._secrets['login_password'],
            account=self._secrets['account'],
            warehouse=self._secrets['warehouse'],
            database=dbname,
            schema=schema
        )

        return ctx

/home/ec2-user/.local/lib/python3.6/site-packages/snowflake/connector/options.py:97: UserWarning: You have an incompatible version of 'pyarrow' installed (4.0.1), please install a version that adheres to: 'pyarrow<5.1.0,>=5.0.0; extra == "pandas"'
  _expected_pyarrow_version,


In [3]:
conn=SnowflakeConnector(SSMPSCredentials(SF_CREDS))
ctx=conn.connect("MAX_PROD","DATASCIENCE_STAGE")
cur = ctx.cursor()
# Execute a statement that will generate a result set.
querystr='''
 SELECT 1
'''
cur.execute(querystr)
# Fetch the result set from the cursor and deliver it as the Pandas DataFrame.


In [ ]:
#df = cur.fetch_pandas_all()

In [ ]:
colstring = ','.join([col[0] for col in cur.description])
df = pd.DataFrame(cur.fetchall(), columns =colstring.split(",")) 